In [ ]:
#all the imports needed for the functions to work
import numpy as np
from numpy.linalg import eig
import itertools as it
from itertools import chain, combinations

In [ ]:
#these functions are useful for iterating over a set of items
def Fp(p):
    Fp = [element for element in range(p)]
    return Fp

def Fnp(n,Fp):
    combi = list(list(it.product(Fp, repeat=n)))
    return combi

In [ ]:
#this removes a given item from a list
def remove_items(test_list, item):
    res = [i for i in test_list if i != item] 
    return res

#this finds the sum of a list of numbers
def listsum(vec):
    num = 0
    for thing in vec:
        num+=thing
    return num

## Functions Manipulating Gates in Pauli and Symplectic Form

The ability to convert gates from their Pauli representation to their symplectic form is very useful.

The base gates and their associated matrices are as follows:

$$X = \begin{pmatrix}0&1\\1&0\end{pmatrix} \;\; Z = \begin{pmatrix}1&0\\0&-1\end{pmatrix} \;\; Y = \begin{pmatrix}0&-i\\i&0\end{pmatrix}$$

The Pauli representation is the gates' most typical form. An example of such representation is $X \otimes I \otimes Z \otimes Y$. This shows that an $X$ gate acts on the first qubit, the second qubit is unchanged, a $Z$ gate acts on the third qubit, and $Y$ acts on the fourth qubit. Usually when represented in this way the tensor product notation is collapsed on, such that the gates would be instead written as $XIZY$. Noting that $(X)(Z) = iY$, these gates may also be represented by the following equation:

$$\mathcal{P} = (-1)^c(-i)^{\vec{q}\cdot\vec{p}}(X)^\vec{q}(Z)^\vec{p}$$

The symplectic representation of quantum gates is another way of describing whether an X and/or Z gate is acting on a qubit. It uses the equation above and hilights just the vectors $\vec{q},\vec{p}$ and $c$. For instance the example gate above may be rewritten in its symplectic form as:

$$XIZY \rightarrow \left(--\vec{q}--|--\vec{p}--|c\right) = \left(1,0,0,1|0,0,1,1|0\right)$$

The advantage of this representation is multiplication of gates becomes addition of their symplectic binary strings:
$$\begin{align*}
(X ⊗ I ⊗ Z ⊗ Y )(−X ⊗ Y ⊗ X ⊗ X) &= [1, 0, 0, 1|0, 0, 1, 1|0]\\
&+ [1, 1, 1, 1|0, 1, 0, 0|1]\\
&= [0, 1, 1, 0|0, 1, 1, 1|1]\\
&= −I ⊗ Y ⊗ Y ⊗ Z
\end{align*}$$

It should be noted that the addition is only perfect if the twisted inner product of the two gates is equal to zero:

$$g_1 \odot g_2 = \vec{q}_1\cdot\vec{p}_2 \oplus \vec{q}_2\cdot\vec{p}_1 = 0$$

This ensures there are no unwanted imaginary phases, and hence why we may regard the phase bit as only being binary and not quaternary.

With that information in mind, we needed to create some functions that could do the following:
* Convert a gate from its Pauli representation to its symplectic form
* Multiply two sumplectic gates together
* Keep the phases of the multiplied gates in check
* Convert a gate from its symplectic form to its Pauli representation

In [ ]:
#this function converts a gate from Pauli form to symplectic
#it expects the gate in a string form
def symplectify(gate):
    gate = list(gate)
    
    minus=False
    imag =False
    
    if "-" in gate:               #checks if the gate has a minus phase
        index = gate.index("-")   #also removes the minus such that the symplectic form is the right shape
        gate.pop(index)
        minus=True
    if "i" in gate:               #checks if the gate has an imaginary phase
        index = gate.index("i")   #also removes the imaginary number such that the symplectic form is the right shape
        gate.pop(index)
        imag=True
    
    form = [np.zeros(len(gate)),np.zeros(len(gate)),np.array([0])]        #set up the shape of the symplectic form
    
    if minus == True:               #this governs the phase bit of the symplectic form
        form[2][0] += 1
    if imag == True:                #later on our gates should never have an imaginary phase, -
        print("Unwanted phase")     #- so if we have one we have an issue
    
    for S in gate:                  #here we run over every tensored gate
        if S == "X":                #if the gate is an "X", the symplectic form's first column is updated
            index = gate.index(S)
            gate[index] = "done"
            form[0][index] += 1

        if S == "Z":                #if the gate is a "Z", the symplectic form's second column is updated
            index = gate.index(S)
            gate[index] = "done"
            form[1][index] += 1

        if S == "Y":                #if the gate is a "Y", the symplectic form's first and second columns are updated
            index = gate.index(S)
            gate[index] = "done"
            form[0][index] += 1
            form[1][index] += 1
            
    return form                     #returns the gate's symplectic form



#this function calculates the phase change produced when you multiply two gates together
def g_(a,b,c,d):
    phase=0
    for i in range(len(a)):
        phase += a[i]**2*b[i]**2 + c[i]**2*d[i]**2 - (a[i]+c[i])**2*(b[i]+d[i])**2 + 2*c[i]*b[i]
    return phase%4



#this function multiplies two gates together
#it expects the gates in the symplectic form
def symp_mul(gate1, gate2):
    new_gate = [np.zeros(len(gate1[0])),np.zeros(len(gate1[0])),np.array([0])]
    new_gate[0] = (gate1[0] + gate2[0])%2
    new_gate[1] = (gate1[1] + gate2[1])%2
    new_gate[2] = ((gate1[2]*2 + gate2[2]*2 + g_(gate1[0],gate1[1],gate2[0],gate2[1]))%4)/2
    
    return new_gate



#this function converts the gate from symplectic form to Pauli form
#it expects the gate to be in the symplectic form
def Paulify(symp_gate):
    Pgate = []
    for i in range(len(symp_gate[0])):
        if symp_gate[0][i] == 1 and symp_gate[1][i] == 1: #if theres a 1 in both q and p, the ith gate is Y
            Pgate.append("Y")
        elif symp_gate[0][i] == 1:                        #elif theres a 1 in q only, the ith gate is X
            Pgate.append("X")
        elif symp_gate[1][i] == 1:                        #elif theres a 1 in p only, the ith gate is Z
            Pgate.append("Z")
        else:                                             #else the gate must be I
            Pgate.append("I")
            
    Pgate = "".join(Pgate)
    
    if symp_gate[2][0]==1:                                #if the phase bit is 1 then the gate must be negative
        Pgate = "-" + Pgate
        
    return Pgate

## Functions Needed to Calculate States from Affine Subspace Triple

The equation used to find a stabilizer state from an AST is:

$$\left|\mathcal{K},q,l\right\rangle = \left|\Psi\right\rangle \propto \sum_{x \in \mathcal{K}}i^{l(x)}(-1)^{Q(x)}\left|x\right\rangle$$

Where:
* $\left|\Psi\right\rangle$ is a stabilizer state
* $\mathcal{L(K)}$ is a dimension $k$ linear subspace, $\mathcal{L(K)} \subseteq \mathbb{F}^{n}_{2}$, with basis $\{g^1,g^2,...,g^k\}$
* $\mathcal{K}$ is a dimension $k$ affine subspace, $\mathcal{K} \subseteq \mathbb{F}^{n}_{2}$, with $\mathcal{K} = \mathcal{L(K)} \oplus \vec{h}$
* $\vec{h}$ is a nonunique shift vector, $\vec{h} \in \mathcal{K}$
* $x \in \mathcal{K}$ is a point in the affine space, defined by a coordinate vector $\vec\alpha = (\alpha_1,\alpha_2,...,\alpha_k)$ as $x = \sum_{i=1}^k\alpha_ig^i$
* $l(x)$ is a linear form that maps the affine space to a binary value; $l(x): \mathcal{K}\mapsto \mathbb{F}_2$. It is defined by a binary valued $1\times k$ vector $\vec{b}$ as $l(x) = \vec{b}\cdot\vec{\alpha}$
* $Q(x)$ is a quadratic form that maps the affine space to a binary value; $Q(x): \mathcal{K}\mapsto \mathbb{F}_2$. It is defined by a binary valued $k\times k$ upper triangular matrix $J$ as $Q(x) = \vec{\alpha}J\vec{\alpha}^T$

Considering the above information, it was necessary that we created a function that could compute the above equation. In doing so, some extra helper equations were created:
* A function that finds the generators of any input space
* Functions that calculated the values of $l(x)$ and $Q(x)$ for given $\vec{b}$ and $J$ respectively
* The function that creates a dictionary of all the computational basis vectors of a stabilizer state of a given $n$
* A function that can find all possible $J$ matrices for a given $n$

In [ ]:
#this function finds the generators of any linear subspace inputted
def gen_finder(LK):
    
    n = len(LK[0])      #determines the number of qubits we are working with
    store1 = []         #store1 is going to be our output list of generators
    store2 = [(0,)*n]   #store2 is going to try to recreate the linear subspace to make sure we -
                        #- only have linearly independant vectors in store1
    
    for i in range(len(LK)):        #we loop over each vector in the linear subspace
        if tuple(LK[i]) in store2:  #check if the vector is already inside store2
            continue
        store1.append(LK[i])        #if it isn't yet inside store2 it is appended to store1
        
        for j in range(len(store2)):     #we then must loop over each vector in store2
            vec = np.array(store2[j])
            vec2 = tuple((LK[i]+vec)%2)  #create a new vector by summing the vector just added to store1 and each -
                                         #- vector in store2
            if vec2 not in store2:       #if the new vector isn't in store2 it is added to store2
                store2.append(vec2)
            #this basically regenerates the linear subspace, and makes sure all vectors in store1 are linearly independant
    return(store1)     #return the store1, which should only contain the generators of the linear subspace



#this function evaluates the linear form l(z)
def l(b,α):
    if len(α)==0:
        return 0
    return np.dot(b,α)



#this function evaluates the quaratic form Q(z)
def Q(J,α):
    k = len(α)
    if k==0:
        return 0
    
    α = α[np.newaxis]
    αJ = np.dot(α,J)
    αJαT = np.dot(αJ,α.T)
    
    return αJαT



#this function creates a dictionary linking all possible state names and its correspinding index
#in state vector form for a given n
def state_dictmaker(n):
    state_names = []
    for tup in Fnp(n,["0", "1"]):
        name = ""
        for s in tup:
            name+=s
        state_names.append(name)
    
    state_dict = {}
    for i in range(len(state_names)):
        state_dict[state_names[i]] = i
    
    return state_dict



def AST_to_stab2(gs, h, b, J,state_dict):
    k = len(b)                      #find the dimension of the linear subspace
    n = len(h)                      #find the number of qubits of the stabilizer state
    
    if k == 0:                      #create the linear subspace
        LK = [np.array([0]*n)]      #some of the functions don't work for values of zero, so its easier to make exceptions
    else:
        LK = []                     #use the generators to create the linear subspace
        alphas = Fnp(k,Fp(2))
        for alpha in alphas:
            vec = np.array([0]*n)
            for i in range(len(alpha)):
                vec+=np.dot(alpha[i],np.array(gs[i]))
            LK.append(vec)
    
    K = [list((v+h)%2) for v in LK] #create the affine subspace by adding the shift vector to each vector in the linear subspace
    
    vec_index = []                  #find the indices of the state vector to input values to
    for vec in K:
        thing = ""
        for s in vec:
            thing+=str(s)
        vec_index.append(state_dict[thing])
    
    αs = Fnp(k,Fp(2))               #generate the vector coordinates that will simply correspond in an orderly manner with -
    for i in range(len(αs)):        #- the x points in the subspace K
        αs[i] = np.array(αs[i])
    
    vec = [0]*2**n                  #initialise the state vector 
    tidy_num = []                   #start a list to append the phases of the basis vectors to
    if k==0:                        #another exception for k=0
        tidy_num.append(1)
        
    for α in αs:                             #loop over all α vectors, which correspond one to one with the x points in K
        num = (-1)**(Q(J,α))*(1j)**(l(b,α))  #calculate the number
        tidy_num.append(num)                 #append it to the list of phases
        
    for i in vec_index:             #set the appropriate values in the vector to those found in tidy_num
        vec[i] = tidy_num.pop(0)
        
    return vec                      #return the output



def Js_finder(k):             #this function finds all possible J matrices for the quadratic form for a given n
    if k == 0:                #an exception for k=0
        return [[[]]]
    
    cnum = int(0.5*k*(k+1))   #find the number of different binary combinations possible for J
    cs = [list(c) for c in Fnp(cnum,Fp(2))]  #generate the different binary combinations that will be inserted into J

    Js = []                   #initialise the list of Js
    for c in cs:
        J = [[0 for _ in range(k)] for _ in range(k)]   #for each binary combo c, a new J is initialised
        for i in range(len(J)):                         
            for j in range(len(J[i])):
                if i<=j:                                #only the upper diagonal entries of J are populated
                    J[i][j] = c.pop(0)
        Js.append(J)
    return Js                 #return the list of Js

## States from Check Matrix 

A check matrix is a $n\times 2n+1$ matrix, which shows the symplectic form of the basis gates of a set of gates that stabilize a stabilizer state $\left|\Psi\right\rangle$. It is best represented in its row reduced echelon form, and as such will resemble the following:
$$\begin{pmatrix}
    --\vec{q}_1--&|&--\vec{p}_1--&|&c_1\\
    --\vec{q}_2--&|&--\vec{p}_2--&|&c_2\\
    .&|&.&|&.\\
    .&|&.&|&.\\
    .&|&.&|&.\\
    --\vec{q}_k--&|&--\vec{p}_k--&|&c_k\\
    --\vec{0}--&|&--\vec{\rho}_1--&|&\gamma_1\\
    .&|&.&|&.\\
    .&|&.&|&.\\
    .&|&.&|&.\\
    --\vec{0}--&|&--\vec{\rho}_{n-k}--&|&\gamma_{n-k}\\
\end{pmatrix}$$

When given a stabilizer states' check matrix we may extract the state via the following equation:

$$\rho_{\left|\Psi\right\rangle} = \left|\Psi\right\rangle\left\langle\Psi\right| \propto \prod_{i=1}^n\left(\mathbb{I} + g_i\right)$$

where:
* $\rho_{\left|\Psi\right\rangle}$ is the density matrix of the stabilizer state $\left|\Psi\right\rangle$, calculated by taking the outer product of the stabilizer state vectors
* $\mathbb{I}$ is the $2^n\times 2^n$ identity matrix
* $g_i$ is the $i^{\text{th}}$ stabilizer gate found in the stabilizer state's check matrix

The stabilizer state can be retrieved by finding the eigenvector of $\rho_{\left|\Psi\right\rangle}$ that corresponds to the only non-zero eigenvalue.

We therefore need a function that can calculate the above density matrix and extract the stabilizer state. For this we need some helper functions that can:
* Find the names and tensor products of all size $n$ gates and their negatives, and place them in a dictionary.
* Generate all the stabilizer gates from a given set of basis gates

In [ ]:
#define all the gate matrices and names for use in the functions
I = np.array([[1,0],
              [0,1]])

X = np.array([[0,1],
              [1,0]])

Z = np.array([[1,0],
              [0,-1]])

Y = np.array([[0,-1j],
              [1j,0]])

base_gate_names = ["I", "X", "Y", "Z"]
base_gate_dict = {"I":I, "-I":-I, "X":X, "-X":-X, "Y":Y, "-Y":-Y, "Z":Z, "-Z":-Z}



#this function creates all possible tensor multiplied gate names for a given n
def gate_namemaker(n):
    gate_names = []
    for tup in Fnp(n,base_gate_names):
        name = ""
        for g in tup:
            name+=g
        gate_names.append(name)
        gate_names.append("-"+name)
        
    return gate_names



#this function creates all possible tensor multiplied gates in matrix form for a given n
def gate_matmaker(n):
    gate_mats = []
    for tup in Fnp(n,base_gate_names):

        prod1 = [1]
        prod2 = [-1]
        for g in tup:
            prod1 = np.kron(prod1,base_gate_dict[g])
            prod2 = np.kron(prod2,base_gate_dict[g])

        gate_mats.append(prod1)
        gate_mats.append(prod2)
    
    return gate_mats



#this function uses the above two functions to create a dictionary of the tensor multiplied gates names
#correspondng to the matrix form
def gate_dictmaker(n):
    gate_names = gate_namemaker(n)
    gate_mats = gate_matmaker(n)

    gate_dict = {}
    for i in range(len(gate_names)):
        gate_dict[gate_names[i]] = gate_mats[i]

    return gate_dict



#this funtion finds the set of stabilizer gates given a set of basis gates, excluding the Identity gate
def all_gates_mul(basis_gates):
    n = len(basis_gates)
    
    index_list = []                              #unique combinations of the basis gates must be generated for this to work
    for i in range(n):                           #find all the indexes of the gates that will give unique combinations
        for thing in [list(yoke) for yoke in Fnp(i+1,Fp(n))]:
            if len(thing) == len(set(thing)):    #iterate over the indices in a way that will not produce copies
                thing.sort()
                if thing not in index_list:
                    index_list.append(thing)
                    
    final_gate_list = []
    for indexes in index_list:                   #run over all unique sets of indices
        gates_list = []
        for index in indexes:                    #create the unique lists of basis gate combinations
            gates_list.append(basis_gates[index])
        gate_final = "I"*n                       #multiply all the gates in the combination together to generate the full -
        for gate in gates_list:                  #- set of stabilizer gates for the stabilizer state in question
            gate_final = Paulify(symp_mul(symplectify(gate_final), symplectify(gate)))
        final_gate_list.append(gate_final)
        
    return final_gate_list                       #return the set of stabilizer gates



def state_from_gates(n, all_gates, gate_dict, check=True):
    
    mat = gate_dict["I"*n]                 #initialise the matrix
    for gate in all_gates:                 #run through all gates in set of gates that stabilize the state (excluding Identity)
        mat = mat + gate_dict[gate]        #add the gates' matrix form to the overall matrix
    
    if check==True:                        #print matrix if we want to
        print(mat)
    w,v=eig(mat)                           #generate all the eigenvectors and eigenvalues of the matrix
    vect = np.round([v[i][np.argmax(w)] for i in range(len(w))],2) #the stabilizer state will be the eigenvector corresponding -
                                                                   #- to the non-zero eigenvalue of the matrix
    for num in vect:
        if num != 0:                       #This ensures the returned state has the preferred global phase
            thing = num
            break
    
    vect = vect/thing
    return vect

## The AST to Check Matrix Conversion

Below is the current conversion method from AST to check matrix. It is proven to work for all cases up to and including $n=4$.

We may fill in the entries of a row reduced echelon form check matrix via the following rules:

1.  The set $\{\vec{q}_1,\vec{q}_2,..., \vec{q}_k\}$ is the same as the unique row reduced set $\{\vec{g}^1, \vec{g}^2,...,\vec{g}^k\}$, the basis vectors of $\mathcal{L(K)}$. The other $n-k$ entries in the "q-column" are zero vectors $\vec{0}$ of size $1\times n$
2.  $\vec{\rho}_i$ is the basis of the null space of the matrix with rows $\vec{q}$
3.  $\gamma_i = \vec{h}\cdot\vec{\rho}_i$
4.  The set $\{\vec{p}_1,\vec{p}_2,..., \vec{p}_k\}$ is found by creating the $p'$ and $q'$ submatrices. Then the $q'$ submatrix is updated via the steps outlined later, and used to find the entries of $p'$ via equation below. The entries $p'_i$ are related back to those in $\vec{p}_i$, and the values in $\vec{p}_i$ not in $p'_i$ are set to zero.

$$p'_i = b_iq'_i +\sum_{j\neq i}\left(J_{ij}+J_{ji}\right)q'_j$$

5.  The remaining $c$ phases are calculated via equation:
$$c_i = J_{ii} + \vec{p}_i\cdot\vec{h} + \frac{(\vec{p}_i\cdot\vec{q}_i)(\vec{p}_i\cdot\vec{q}_i - 1)}{2} \mod{2}$$

---

The $q'$ submatrix is updated via the following steps:
* The rows of $q'$ are considered. Pairs of rows are created by matching together rows that share ones in the same index, and that contain different amounts of ones.
* Once all the pairs are made, we update the rows of $q'$ by summing the row with the least ones from a pair to its partner.
* Then the row from the pair with the least ones is found, and is summed with the new row created above.

---

The functions below do the following jobs:
* Update the $q'$ submatrix
* Complete the conversion from AST to the gates that the check matrix consist of
* Print the state from the AST equation, the check matrix, and the state from the check matrix
* Check if the set of linear subspace generators work for all possible cases, essentially stress testing the cases

In [ ]:
def q_update(mat):
    mat = np.array(mat)   #make the matrix a numpy array to let us work with it easier
    if mat.size == 0:
        return [[]]       #if the matrix is empty return an empty matrix back
    
    mat2 = mat.copy()     #make a copy of the matrix
    pairs = []            #set up a list we will append pairs of matrix rows to
    pair_indices = []     #make another list to append the indices of the matrix rows above
    
    
    for i in range(len(mat)):                                           #run over each row in the matrix
        for j in range(i+1,len(mat)):                                   #run over each row below the row defined above
            truth = 0                                                   #store a boolean value
            for k in range(len(mat[i])):
                if ((mat[i][k] == 1) and (mat[j][k] == 1)):             #if the rows share a one in the same index, it might -
                    truth = 1                                           #- be a pair, so make the boolean = 1
                    
            if ((truth == 1) and (listsum(mat[i]) != listsum(mat[j]))): #if the boolean=1 and the sum of all values in the -
                pairs.append([mat[i],mat[j]])                           #- pair isnt equal, the pair and its indices is -
                pair_indices.append([i,j])                              #- appended to the lists defined before
    
    max_index = []
    for i in range(len(pairs)):                             #this finds the row in the pair with the most 1 values in it
        thing = [listsum(single) for single in pairs[i]]
        max_index.append(pair_indices[i][np.argmax(thing)])
    
    pair_sums = []
    for pair in pairs:                         #here we find the sums of the pairs mod 2
        vec_sum = list((pair[0] + pair[1])%2)
        pair_sums.append(vec_sum)
    
    for i in range(len(pair_sums)):                            #we run over all the pairs found and update the rows with - 
        mat2[max_index[i]] += mat[max_index[i]] + pair_sums[i] #- the most 1s. not certain this is correct
    
    for i in range(len(pair_indices)):      #the rows in the pair that had the least 1s is updated next
        for j in pair_indices[i]:
            if (j != max_index[i]):
                mat2[j] += mat2[max_index[i]]
                
    return mat2%2     #the updated matrix is returned mod 2



def Check_from_AST(gs, h, b, J, check=True):
    k = len(b)
    n = len(h)
    
    if k == 0:                       #the set of very straightforward exceptions for k=0
        rhos = np.identity(n)
        gamma = np.dot(rhos, h)%2
        qcol = [[0]*n]*(n)
        pcol = list(rhos)
        ccol = list(gamma)
    
    else:
        q = gs.copy()    #qs and gs are the same
        
        nullspace = []                               #initialise the nullspace
        for vec in Fnp(n,Fp(2)):                     #check every vector in the space F^n_2
            vec = np.array(vec)
            vec2 = vec[np.newaxis]
            qvecT = np.dot(q,vec2.T)%2               #find the result of the matrix multiplication
            qvecT = tuple(qvecT)
            if qvecT == tuple([np.array([0])]*k):    #if the result is all zeros, the vector is added to the nullspace list
                nullspace.append(vec)

        if len(nullspace) == 0:            #an exception for the case of k=n
            rhos = []
        else:                              #find the basis of the nullspace
            rhos = gen_finder(nullspace)
            
        rhos = [list(thing) for thing in rhos]    #make sure the rhos are in the correct order
        rhos.reverse()
        
        if len(rhos) == 0:              #an exception for the case of k=n
            gamma = []
        else:                           
            gamma = np.dot(rhos, h)%2   #find all gammas

        bad_index = []                  #initialise the list of indexes that contian a leading one in a rho vector
        for rho in rhos:
            for i in range(len(rho)):
                if rho[i] == 1:         #this finds the first instance of a one in each rho vector
                    break
            bad_index.append(i)         #the index is added to the list "bad_index"

        q_dash = []                     #initialise the q' submatrix
        for q_i in q:                   #run through the k q vectors 
            q_i = list(q_i)
            for j in bad_index:         #assign the bad_index with a dummy constant that would never appear naturally
                q_i[j] = 8
            q_i = remove_items(q_i, 8)  #remove the dummy constant
            q_dash.append(q_i)          #append the new row to the q' submatrix
            
        S = q_update(q_dash)            #update the q' submatrix

        p_dash = []                            #initialise the p' submatrix
        for i in range(k):
            p_dash_i = b[i]*np.array(S[i])     #calculate its entries according to the equation detailed in the description
            for j in range(k):
                if j==i:
                    continue
                p_dash_i += (J[i][j] + J[j][i])*np.array(S[j])
            p_dash.append((p_dash_i)%2)        #append the row to the matrix, making sure it is mod 2

        p = []                          #initialise the p vectors
        for p_dash_i in p_dash:         #run through the p' matrix rows
            p_dash_i = list(p_dash_i)
            for j in bad_index:
                p_dash_i.insert(j,0)   #ensure entries of p that line up with leading ones in the rho vectors are set to zero
            p.append(p_dash_i)      
            
        c = []                         #initialise the c phase bits
        for i in range(k):             #find the values of c via the predefined equation
            c_i = (J[i][i] + np.dot(p[i],h) + np.dot(p[i],q[i])*(np.dot(p[i],q[i])-1)*0.5)%2
            c.append(int(c_i))

        qcol = list(q)+[[0]*n]*(n-k)   #set up the columns of the check matrix
        pcol = p+list(rhos)
        ccol = c + list(gamma)
    
    if check==True:                    #if we wish we may print the check matrix
        for i in range(len(qcol)):
            print(np.array(qcol[i]), np.array(pcol[i]), [ccol[i]])

    gate_list = []                    #find the Pauli representation of the basis gates found in the check matrix
    for i in range(len(qcol)):
        gate_list.append(Paulify([list(qcol[i]), list(pcol[i]), [ccol[i]]]))

    return gate_list                  #return the basis gates

In [ ]:
def result_printer(gs,hs,bs,Js):
    n = len(hs[0])
    gate_dict = gate_dictmaker(n)          #generate the dictionary of all gates possible for given n
    state_dict = state_dictmaker(n)        #generate the dictionary of all computational basis states for a given n
    
    for h in hs:                           #run through all h vectors in the set 
        for J in Js:                       #run through all possible J matrices for the quadratic form
            for b in bs:                   #run through all possible b vectors for the linear form
                print("State from AST:")
                state = AST_to_stab2(gs, h, b, J,state_dict)     #find the state according to the AST equation
                print(state)
                
                print()
                print("Check Matrix:")                           #convert from AST to check matrix
                gates = Check_from_AST(gs, h, b, J)              #find all basis gates found in the check matrix
                
                print()
                print("State from Check Matrix:")
                all_gates = all_gates_mul(gates)                 #find all gates that stabilize the stabilizer state
                state_ = state_from_gates(n,all_gates,gate_dict, check=True)
                print(state_)                                    #calculate the state from the state's density matrix
                
                print()                                          #ascertain if the states are the same
                print("We have same state:", tuple(state) == tuple(state_))
                print("_______________")
                print()
                
def result_validator(gs,hs,bs,Js):
    n = len(hs[0])
    gate_dict = gate_dictmaker(n)          #generate the dictionary of all gates possible for given n
    state_dict = state_dictmaker(n)        #generate the dictionary of all computational basis states for a given n
    truth = 1                              #initialise a boolean value
    for h in hs:                           #run through all h vectors in the set 
        for J in Js:                       #run through all possible J matrices for the quadratic form
            for b in bs:                   #run through all possible b vectors for the linear form
                state = AST_to_stab2(gs, h, b, J,state_dict)     #find the state according to the AST equation
                gates = Check_from_AST(gs, h, b, J,check=False)  #find all basis gates found in the check matrix
                all_gates = all_gates_mul(gates)                 #find all gates that stabilize the stabilizer state
                state_ = state_from_gates(n,all_gates,gate_dict,check=False)#calculate the state from the state's density matrix

                if (tuple(state) == tuple(state_)) == False:     #ascertain if the states are the same
                    truth = 0                                    #if even one issue arises, we will know that the set of -
                                                                 #- generators must be looked at more closely
                    break
                
    print("gs", gs, "works:", bool(truth))

## $n=1, k=0$

No generator, $g\in \{\}$. Only thing that changes is the shift vector $\vec{h}$, giving a total of 2 stabilizer states.

In [ ]:
gs = [[]]
hs = [[0], [1]]
bs = [list(v) for v in Fnp(0,Fp(2))]
Js = Js_finder(0)

result_printer(gs,hs,bs,Js)

## $n=1, k=1$

One generator, $g\in \{1\}$. Shift vector doesn't change, $\vec{h}=(0)$, but there are two different quadratic forms and two different linear forms, giving a total of 4 stabilizer states.

In [ ]:
gs = [[1]]
hs = [[0]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_printer(gs,hs,bs,Js)

## $n=2, k=0$

No generator, $g\in \{\}$. There are four different shift vectors, so 4 different stabilizer states.

In [ ]:
gs = [[]]
hs = [[0,0], [0,1], [1,0], [1,1]]
bs = [list(v) for v in Fnp(0,Fp(2))]
Js = Js_finder(0)

result_printer(gs,hs,bs,Js)

## $n=2, k=1, g=(0,1)$

$\vec{h}$ can be $(0,0)$ or $(1,0)$, and there are two different linear and quadratic forms each, resulting in 8 stabilizer states.

In [ ]:
gs = [[0,1]]
hs = [[0,0],[1,0]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_printer(gs,hs,bs,Js)

## $n=2, k=1, g=(1,0)$

Similar to above, $\vec{h}$ can be $(0,0)$ or $(0,1)$, and there are two different linear and quadratic forms each, resulting in 8 stabilizer states.

In [ ]:
gs = [[1,0]]
hs = [[0,0],[0,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=2, k=1, g=(1,1)$

Similar to above, $h$ can be $(0,0)$ or $(0,1)$, and there are two different linear and quadratic forms each, resulting in 8 stabilizer states.

In [ ]:
gs = [[1,1]]
hs = [[0,0],[0,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=2, k=2$

One set of generators, $g \in \{(0,1),(1,0)\}$. $\vec{h}$ doesn't change from $(0,0)$. There are four different linear forms, and 8 quadratic forms, giving 32 stabilizer states in total.

In [ ]:
gs = [[1,0],[0,1]]
hs = [[0,0]]
bs = [list(v) for v in Fnp(2,Fp(2))]
Js = Js_finder(2)

result_validator(gs,hs,bs,Js)

## $n=3, k=0$

No generator, $g\in \{\}$. Only thing that changes is the shift vector $\vec{h}$, of which there are 8, giving 8 different states

In [ ]:
gs = [[]]
hs = [[0,0,0], [0,0,1], [0,1,0], [0,1,1],[1,0,0], [1,0,1], [1,1,0], [1,1,1]]
bs = [list(v) for v in Fnp(0,Fp(2))]
Js = Js_finder(0)

result_validator(gs,hs,bs,Js)

## $n=3, k=1, g=(0,0,1)$

$\vec{h} \in \{(0,0,0), (0,1,0),(1,0,0),(1,1,0)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[0,0,1]]
hs = [[0,0,0],[0,1,0],[1,0,0],[1,1,0]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_printer(gs,hs,bs,Js)

## $n=3, k=1, g=(0,1,0)$

$\vec{h} \in \{(0,0,0), (0,0,1),(1,0,0),(1,0,1)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[0,1,0]]
hs = [[0,0,0],[0,0,1],[1,0,0],[1,0,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=3, k=1, g=(0,1,1)$

$\vec{h} \in \{(0,0,0), (0,0,1),(1,0,0),(1,0,1)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[0,1,1]]
hs = [[0,0,0],[0,0,1],[1,0,0],[1,0,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=3, k=1, g=(1,0,0)$

$\vec{h} \in \{(0,0,0), (0,0,1),(0,1,0),(0,1,1)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[1,0,0]]
hs = [[0,0,0],[0,0,1],[0,1,0],[0,1,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=3, k=1, g=(1,0,1)$

$\vec{h} \in \{(0,0,0), (0,0,1),(0,1,0),(0,1,1)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[1,0,1]]
hs = [[0,0,0],[0,0,1],[0,1,0],[0,1,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=3, k=1, g=(1,1,0)$

$\vec{h} \in \{(0,0,0), (0,0,1),(0,1,0),(0,1,1)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[1,1,0]]
hs = [[0,0,0],[0,0,1],[0,1,0],[0,1,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=3, k=1, g=(1,1,1)$

$\vec{h} \in \{(0,0,0), (0,0,1),(0,1,0),(0,1,1)\}$, two linear forms, two quadratic forms, totals 16 states

In [ ]:
gs = [[1,1,1]]
hs = [[0,0,0],[0,0,1],[0,1,0],[0,1,1]]
bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)

result_validator(gs,hs,bs,Js)

## $n=3, k=2$

theres 7 sets of linear subspaces, each with two shift vectors, four linear forms and eight quadratic forms:
* $g\in \{(0,0,1),(0,1,0)\},\;\; h \in \{(0,0,0),(1,0,0)\}$
* $g\in \{(0,0,1),(1,0,0)\},\;\; h \in \{(0,0,0),(0,1,0)\}$
* $g\in \{(0,0,1),(1,1,0)\},\;\; h \in \{(0,0,0),(0,1,0)\}$
* $g\in \{(0,1,0),(1,0,0)\},\;\; h \in \{(0,0,0),(0,0,1)\}$
* $g\in \{(0,1,0),(1,0,1)\},\;\; h \in \{(0,0,0),(0,0,1)\}$
* $g\in \{(0,1,1),(1,0,0)\},\;\; h \in \{(0,0,0),(0,0,1)\}$
* $g\in \{(0,1,1),(1,0,1)\},\;\; h \in \{(0,0,0),(0,0,1)\}$

Total number of stabilizer states is 448

In [ ]:
gs_list = [[[0,1,0],[0,0,1]],
           [[1,0,0],[0,0,1]],
           [[1,1,0],[0,0,1]],
           [[1,0,0],[0,1,0]],
           [[1,0,1],[0,1,0]],
           [[1,0,0],[0,1,1]],
           [[1,0,1],[0,1,1]]]

hs_list = [[[0,0,0],[1,0,0]],
           [[0,0,0],[0,1,0]],
           [[0,0,0],[0,1,0]],
           [[0,0,0],[0,0,1]],
           [[0,0,0],[0,0,1]],
           [[0,0,0],[0,0,1]],
           [[0,0,0],[0,0,1]]]

bs = [list(v) for v in Fnp(2,Fp(2))]
Js = Js_finder(2)


for gen_index in range(len(gs_list)):
    result_validator(gs_list[gen_index],hs_list[gen_index],bs,Js)

## $n=3, k=3$

One linear subspace; $g\in \{(0,0,1),(0,1,0),(1,0,0)\},\;\; \vec{h} = (0,0,0)$, but there's eight different linear forms and 64 quadratic forms, giving us 512 stabilizer states.

In [ ]:
gs = [[1,0,0],[0,1,0],[0,0,1]]
hs = [[0,0,0]]
bs = [list(v) for v in Fnp(3,Fp(2))]
Js = Js_finder(3)

result_validator(gs,hs,bs,Js)

## $n=4, k=0$

theres only one linear subsace; $g\in \{\}$, each with 16 shift vectors: 16 total stabilizer states

In [ ]:
gs = [[]]
hs = [list(v) for v in Fnp(4,Fp(2))]
bs = [list(v) for v in Fnp(0,Fp(2))]
Js = Js_finder(0)

result_printer(gs,hs,bs,Js)

In [ ]:
gs = [[]]
hs = [list(v) for v in Fnp(4,Fp(2))]
bs = [list(v) for v in Fnp(0,Fp(2))]
Js = Js_finder(0)

result_validator(gs,hs,bs,Js)

## $n=4, k=1$

15 linear subspaces, each with 8 shift vectors, 2 linear forms and two quadratic forms: 480 total

In [ ]:
gs_list = [[[0,0,0,1]],[[0,0,1,0]],[[0,0,1,1]],[[0,1,0,0]],
           [[0,1,0,1]],[[0,1,1,0]],[[0,1,1,1]],[[1,0,0,0]],
           [[1,0,0,1]],[[1,0,1,0]],[[1,0,1,1]],[[1,1,0,0]],
           [[1,1,0,1]],[[1,1,1,0]],[[1,1,1,1]]]
           

hs_list = [[[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,1,1,0],[1,0,0,0],[1,0,1,0],[1,1,0,0],[1,1,1,0]],
           [[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1],[1,0,0,0],[1,0,0,1],[1,1,0,0],[1,1,0,1]],
           [[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1],[1,0,0,0],[1,0,0,1],[1,1,0,0],[1,1,0,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[1,0,0,0],[1,0,0,1],[1,0,1,0],[1,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[1,0,0,0],[1,0,0,1],[1,0,1,0],[1,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[1,0,0,0],[1,0,0,1],[1,0,1,0],[1,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[1,0,0,0],[1,0,0,1],[1,0,1,0],[1,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1],[0,1,0,0],[0,1,0,1],[0,1,1,0],[0,1,1,1]]]

bs = [list(v) for v in Fnp(1,Fp(2))]
Js = Js_finder(1)


for gen_index in range(len(gs_list)):
    result_validator(gs_list[gen_index],hs_list[gen_index],bs,Js)

## $n=4, k=2$

theres 35 linear subspaces, each with 4 shift vectors, 4 linear forms, 8 quadratic forms: 4480 total

In [ ]:
gs_list = [[[0, 0, 1, 0], [0, 0, 0, 1]],[[0, 1, 0, 0], [0, 0, 0, 1]],[[0, 1, 1, 0], [0, 0, 0, 1]],[[1, 0, 0, 0], [0, 0, 0, 1]],
           [[1, 0, 1, 0], [0, 0, 0, 1]],[[1, 1, 0, 0], [0, 0, 0, 1]],[[1, 1, 1, 0], [0, 0, 0, 1]],
           [[0, 1, 0, 0], [0, 0, 1, 0]],[[0, 1, 0, 1], [0, 0, 1, 0]],[[1, 0, 0, 0], [0, 0, 1, 0]],[[1, 0, 0, 1],[0, 0, 1, 0]],
           [[1, 1, 0, 0], [0, 0, 1, 0]],[[1, 1, 0, 1], [0, 0, 1, 0]],
           [[0, 1, 0, 0], [0, 0, 1, 1]],[[0, 1, 0, 1], [0, 0, 1, 1]],[[1, 0, 0, 0], [0, 0, 1, 1]],[[1, 0, 0, 1], [0, 0, 1, 1]],
           [[1, 1, 0, 0], [0, 0, 1, 1]],[[1, 1, 0, 1], [0, 0, 1, 1]],
           [[1, 0, 0, 0], [0, 1, 0, 0]],[[1, 0, 0, 1], [0, 1, 0, 0]],[[1, 0, 1, 0], [0, 1, 0, 0]],[[1, 0, 1, 1], [0, 1, 0, 0]],
           [[1, 0, 0, 0], [0, 1, 0, 1]],[[1, 0, 0, 1], [0, 1, 0, 1]],[[1, 0, 1, 0], [0, 1, 0, 1]],[[1, 0, 1, 1], [0, 1, 0, 1]],
           [[1, 0, 0, 0], [0, 1, 1, 0]],[[1, 0, 0, 1], [0, 1, 1, 0]],[[1, 0, 1, 0], [0, 1, 1, 0]],[[1, 0, 1, 1], [0, 1, 1, 0]],
           [[1, 0, 0, 0], [0, 1, 1, 1]],[[1, 0, 0, 1], [0, 1, 1, 1]],[[1, 0, 1, 0], [0, 1, 1, 1]],[[1, 0, 1, 1], [0, 1, 1, 1]]]
           

hs_list = [[[0,0,0,0],[0,1,0,0],[1,0,0,0],[1,1,0,0]],[[0,0,0,0],[0,0,1,0],[1,0,0,0],[1,0,1,0]],
           [[0,0,0,0],[0,0,1,0],[1,0,0,0],[1,0,1,0]],[[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,1,1,0]],
           [[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,1,1,0]],[[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,1,1,0]],
           [[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,1,1,0]],[[0,0,0,0],[0,0,0,1],[1,0,0,0],[1,0,0,1]],
           [[0,0,0,0],[0,0,0,1],[1,0,0,0],[1,0,0,1]],[[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],
           [[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],[[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],
           [[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],[[0,0,0,0],[0,0,0,1],[1,0,0,0],[1,0,0,1]],
           [[0,0,0,0],[0,0,0,1],[1,0,0,0],[1,0,0,1]],[[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],
           [[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],[[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],
           [[0,0,0,0],[0,0,0,1],[0,1,0,0],[0,1,0,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],[[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]],
           [[0,0,0,0],[0,0,0,1],[0,0,1,0],[0,0,1,1]]]

bs = [list(v) for v in Fnp(2,Fp(2))]
Js = Js_finder(2)


for gen_index in range(len(gs_list)):
    result_validator(gs_list[gen_index],hs_list[gen_index],bs,Js)

## $n=4, k=3$

15 linear subspaces with 2 shift vectors each, 8 linear forms, 64 quadratic forms: 15360 total

In [ ]:
gs_list = [[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
           [[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
           [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]],
           [[1, 0, 1, 0], [0, 1, 1, 0], [0, 0, 0, 1]],
           [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]],
           [[1, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0]],
           [[1, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 1]],
           [[1, 0, 0, 0], [0, 1, 0, 1], [0, 0, 1, 1]],
           [[1, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1]],
           [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 1]],
           [[1, 0, 0, 0], [0, 1, 0, 1], [0, 0, 1, 0]],
           [[1, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]],
           [[1, 0, 0, 0], [0, 1, 1, 0], [0, 0, 0, 1]],
           [[1, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]],
           [[1, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]]
           

hs_list = [[[0,0,0,0],[1,0,0,0]],
           [[0,0,0,0],[0,1,0,0]],
           [[0,0,0,0],[0,0,1,0]],
           [[0,0,0,0],[0,0,1,0]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,0,1]],
           [[0,0,0,0],[0,0,1,0]],
           [[0,0,0,0],[0,0,1,0]],
           [[0,0,0,0],[0,1,0,0]],]

bs = [list(v) for v in Fnp(3,Fp(2))]
Js = Js_finder(3)


for gen_index in range(len(gs_list)):
    result_validator(gs_list[gen_index],hs_list[gen_index],bs,Js)

## $n=4, k=4$
Only one linear subspace and one shift vector, 16 linear forms and 1024 quadratic forms: 16384 total stabilizer states

In [ ]:
gs = [[1,0,0,0],[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
hs = [[0,0,0,0]]
bs = [list(v) for v in Fnp(4,Fp(2))]
Js = Js_finder(4)

result_validator(gs,hs,bs,Js)